In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
train_identity = pd.read_csv('/home/other/kaggle/data/train_identity.csv')
train_identity.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [3]:
# for col in train_identity.columns:
#     if type(train_identity[col].iloc[0]) == np.float64:
#         train_identity[col] = train_identity[col].astype(np.float16)
# #train_identity.to_csv('/kaggle/low_dim/train_transaction.csv')

In [4]:
train_transaction = pd.read_csv('/home/other/kaggle/data/train_transaction.csv')
train_transaction.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# for col in train_transaction.columns:
#     if type(train_transaction[col].iloc[0]) == np.float64:
#         train_transaction[col] = train_transaction[col].astype(np.float16)
# train_transaction.to_csv('/kaggle/low_dim/train_transaction.csv')

In [6]:
target = 'isFraud'

In [7]:
train_identity.shape, train_transaction.shape

((144233, 41), (590540, 394))

In [8]:
df = pd.merge(train_identity, train_transaction, on='TransactionID', how='outer')
df.shape

(590540, 434)

In [9]:
# drop columns where more than 50% of NaNs

drop_cols = df.isna().sum()[df.isna().sum() > (df.shape[0] / 2)].index.tolist()
df.drop(drop_cols, axis=1, inplace=True)
df.shape

(590540, 220)

In [10]:
cat_cols_transaction = ['ProductCD',  
                        'addr1', 
                        'addr2',
                        'P_emaildomain',
                        'R_emaildomain']
cat_cols_transaction.extend(['card{}'.format(i) for i in range(1, 7)])
cat_cols_transaction.extend(['M{}'.format(i) for i in range(1, 10)])

cat_cols_identity = ['DeviceType',
                     'DeviceInfo']
cat_cols_identity.extend(['id_{}'.format(i) for i in range(12, 39)])

print('cat_cols_transaction: ', cat_cols_transaction)
print('cat_cols_identity: ', cat_cols_identity)

cat_cols_transaction:  ['ProductCD', 'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9']
cat_cols_identity:  ['DeviceType', 'DeviceInfo', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']


In [11]:
cat_cols_tr = []
for col in cat_cols_transaction:
    if col not in drop_cols:
        cat_cols_tr.append(col)

cat_cols_id = []
for col in cat_cols_identity:
    if col not in drop_cols:
        cat_cols_id.append(col)
        
print('cat_cols_tr: ', cat_cols_tr)
print('cat_cols_id: ', cat_cols_id)

cat_cols_tr:  ['ProductCD', 'addr1', 'addr2', 'P_emaildomain', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'M1', 'M2', 'M3', 'M4', 'M6']
cat_cols_id:  []


In [12]:
# for col in cat_cols_id:
#     if df[col].nunique() < 3:
#         print(col, df[col].unique())

In [13]:
for col in cat_cols_tr:
    if df[col].nunique() < 3:
        print(col, df[col].unique())

M1 [nan 'T' 'F']
M2 [nan 'T' 'F']
M3 [nan 'T' 'F']
M6 [nan 'T' 'F']


In [14]:
# train_identity[cat_cols_id].fillna(-1, inplace=True)
df[cat_cols_tr].fillna(-1, inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4323: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [15]:
# not_cat_cols_id

In [16]:
# cat_cols_id

In [17]:
# not_cat_cols_id = [col for col in df.columns if col not in cat_cols_id and col != target]
not_cat_cols_tr = [col for col in df.columns if col not in cat_cols_tr and col != target]

In [18]:
# df[not_cat_cols_id].apply(lambda x: x.fillna(x.values.mean(), inplace=True))
df[not_cat_cols_tr].apply(lambda x: x.fillna(x.values.mean(), inplace=True))

TransactionID     None
TransactionDT     None
TransactionAmt    None
C1                None
C2                None
                  ... 
V317              None
V318              None
V319              None
V320              None
V321              None
Length: 204, dtype: object

In [19]:
df['V317'].unique()

array([   0.        ,  100.        ,   48.63809967, ..., 3816.        ,
        680.5       , 1908.        ])

TransactionDT - timedelta. Suggest that it is the number of seconds since 1st January 2017

In [20]:
df['timestamp'] = pd.to_datetime(df['TransactionDT'], unit='s')
df[['TransactionDT', 'timestamp']]

,TransactionDT,timestamp
0,86506,1970-01-02 00:01:46
1,86535,1970-01-02 00:02:15
2,86549,1970-01-02 00:02:29
3,86555,1970-01-02 00:02:35
4,86620,1970-01-02 00:03:40
...,...,...
590535,15811047,1970-07-02 23:57:27
590536,15811049,1970-07-02 23:57:29
590537,15811079,1970-07-02 23:57:59
590538,15811088,1970-07-02 23:58:08


In [21]:
df['timestamp'].iloc[0].year

1970

In [22]:
for x in df['timestamp']:
    if x.year != 1970:
        print(x)

All rows are belonged to one year

In [23]:
start_date = df['timestamp'].iloc[0].date()
i = 0 
j = 0
date_month_agg = []
while i < len(df):
    #print(df['timestamp'].iloc[i].date(), start_date, df['timestamp'].iloc[i].date() == start_date)
    if df['timestamp'].iloc[i].date() == start_date:
        date_month_agg.append(j)
    else:
        j += 1
        start_date = df['timestamp'].iloc[i].date()
        date_month_agg.append(j)
    i += 1
len(date_month_agg) == len(df)

True

In [24]:
df['date_month_agg'] = date_month_agg
df[['timestamp', 'date_month_agg']]

,timestamp,date_month_agg
0,1970-01-02 00:01:46,0
1,1970-01-02 00:02:15,0
2,1970-01-02 00:02:29,0
3,1970-01-02 00:02:35,0
4,1970-01-02 00:03:40,0
...,...,...
590535,1970-07-02 23:57:27,363
590536,1970-07-02 23:57:29,363
590537,1970-07-02 23:57:59,363
590538,1970-07-02 23:58:08,363


In [25]:
# Encoding of categorical features

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df[cat_cols_tr] = df[cat_cols_tr].apply(lambda x: le.fit_transform(x.astype(str)))

In [26]:
df[cat_cols_tr].head()

,ProductCD,addr1,addr2,P_emaildomain,card1,card2,card3,card4,card5,card6,M1,M2,M3,M4,M6
0,1,241,65,16,9262,413,42,2,2,1,2,2,2,3,2
1,1,183,65,1,7962,0,42,4,108,2,2,2,2,3,2
2,0,332,74,16,5591,251,11,2,30,1,2,2,2,0,2
3,0,332,74,19,9234,274,73,2,106,2,2,2,2,0,2
4,1,52,65,2,6791,454,42,4,108,2,2,2,2,3,2


In [27]:
df.drop('timestamp', axis=1, inplace=True)

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.drop(target, axis=1), 
                                                  df[target], 
                                                  test_size=0.33, 
                                                  shuffle=True, 
                                                  stratify=df[target])

In [29]:
X_train.values

array([[3.0196630e+06, 8.1539400e+05, 1.6050000e+02, ..., 0.0000000e+00,
        0.0000000e+00, 1.9000000e+02],
       [3.0477640e+06, 1.3785480e+06, 7.5000000e+01, ..., 0.0000000e+00,
        0.0000000e+00, 1.4000000e+01],
       [3.0774250e+06, 1.8877770e+06, 4.0000000e+01, ..., 0.0000000e+00,
        0.0000000e+00, 2.0000000e+01],
       ...,
       [3.0165230e+06, 7.5191200e+05, 3.1195000e+02, ..., 0.0000000e+00,
        0.0000000e+00, 1.8900000e+02],
       [3.1934710e+06, 4.7449050e+06, 5.0000000e+01, ..., 0.0000000e+00,
        0.0000000e+00, 5.3000000e+01],
       [3.5412320e+06, 1.4658362e+07, 2.8000000e+02, ..., 0.0000000e+00,
        0.0000000e+00, 3.5000000e+02]])

In [30]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

clf = LGBMClassifier()
clf.fit(X_train.values, y_train.values)
proba = clf.predict_proba(X_val.values)[:, 1]

roc_auc_score(y_val, proba)

0.9247878856047431

# Make a prediction for test dataset

In [48]:
test_identity = pd.read_csv('/home/other/kaggle/data/test_identity.csv', names=train_identity.columns)
test_identity.drop(0, axis=0, inplace=True)
test_identity.shape

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,13,14,17,18,19,20,21,22,24,25,26,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(141907, 41)

In [49]:
# for col in test_identity.columns:
#     if type(test_identity[col].iloc[0]) == np.float64:
#         test_identity[col] = test_identity[col].astype(np.float16)
# test_identity.to_csv('/kaggle/low_dim/test_identity.csv')

In [50]:
tr_cols = list(train_transaction.columns)
tr_cols.remove(target)
#tr_cols

In [51]:
test_transaction = pd.read_csv('/home/other/kaggle/data/test_transaction.csv', names=tr_cols)
test_transaction.drop(0, axis=0, inplace=True)
test_transaction.shape

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,1,2,4,5,6,8,10,11,12,13,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,

(506691, 393)

In [52]:
# for col in test_transaction.columns:
#     if type(test_transaction[col].iloc[0]) == np.float64:
#         test_transaction[col] = test_transaction[col].astype(np.float16)
# test_transaction.to_csv('/kaggle/low_dim/test_transaction.csv')

In [53]:
test = pd.merge(test_identity, test_transaction, on='TransactionID', how='right')
test.shape

(506691, 433)

In [54]:
test.drop(drop_cols, axis=1, inplace=True)
test.shape

(506691, 219)

In [55]:
test[cat_cols_tr].fillna(-1, inplace=True)
test[not_cat_cols_tr].apply(lambda x: x.fillna(x.values.astype(np.float32).mean(), inplace=True))

test['timestamp'] = pd.to_datetime(test['TransactionDT'], unit='s')
test[['TransactionDT', 'timestamp']]

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4323: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,TransactionDT,timestamp
0,18403224,1970-08-02 00:00:24
1,18403263,1970-08-02 00:01:03
2,18403310,1970-08-02 00:01:50
3,18403310,1970-08-02 00:01:50
4,18403317,1970-08-02 00:01:57
...,...,...
506686,34214279,1971-01-31 23:57:59
506687,34214287,1971-01-31 23:58:07
506688,34214326,1971-01-31 23:58:46
506689,34214337,1971-01-31 23:58:57


In [56]:
start_date = '1970-07-03'
i = 0 
j = 364
date_month_agg = []
while i < len(test):
    #print(df['timestamp'].iloc[i].date(), start_date, df['timestamp'].iloc[i].date() == start_date)
    if test['timestamp'].iloc[i].date() == start_date:
        date_month_agg.append(j)
    else:
        j += 1
        start_date = test['timestamp'].iloc[i].date()
        date_month_agg.append(j)
    i += 1
len(date_month_agg) == len(test)

True

In [57]:
test['date_month_agg'] = date_month_agg
test[['timestamp', 'date_month_agg']]

,timestamp,date_month_agg
0,1970-08-02 00:00:24,365
1,1970-08-02 00:01:03,365
2,1970-08-02 00:01:50,365
3,1970-08-02 00:01:50,365
4,1970-08-02 00:01:57,365
...,...,...
506686,1971-01-31 23:57:59,547
506687,1971-01-31 23:58:07,547
506688,1971-01-31 23:58:46,547
506689,1971-01-31 23:58:57,547


In [58]:
test[cat_cols_tr] = test[cat_cols_tr].apply(lambda x: le.fit_transform(x.astype(str)))

In [59]:
test.drop('timestamp', axis=1, inplace=True)

In [60]:
test.shape

(506691, 220)

In [61]:
proba_test = clf.predict_proba(test.values)[:, 1]
ans_df = pd.DataFrame(columns=['TransactionID', 'isFraud'])
ans_df['TransactionID'] = test['TransactionID']
ans_df['isFraud'] = proba_test
ans_df

,TransactionID,isFraud
0,3663549,0.027992
1,3663550,0.006342
2,3663551,0.035330
3,3663552,0.002153
4,3663553,0.006720
...,...,...
506686,4170235,0.007904
506687,4170236,0.011818
506688,4170237,0.011201
506689,4170238,0.015952


In [62]:
# ans_df.drop_duplicates('TransactionID', keep='first', inplace=True)
# ans_df

In [63]:
ans_df.to_csv('/home/other/kaggle/submission_1.csv', index=False)

In [47]:
# !kaggle competitions submit -c ieee-fraud-detection -f submission.csv -m "submission_1"